## The test Analysis is performed on the 1st presidential Debate 2020.
The debate transcripts are taken from rev.com

In [1]:
# Import dependencies and required magics

import requests        # import from web
from bs4 import BeautifulSoup      # clean up text
from wordcloud import WordCloud    # create word clouds
from textblob import TextBlob      # basic NLP, install first
from textatistic import Textatistic   # readability, install first
from newspaper import Article
from pathlib import Path    # for quick import of text file for NLP
import nltk
import re
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from plotly import express as px
import plotly.graph_objs as go

# Magics
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

**Transcripts are uploaded from rev.com**

In [2]:
url = 'https://www.rev.com/blog/transcripts/donald-trump-joe-biden-1st-presidential-debate-transcript-2020'

article = Article(url)
article.download()
article.parse()
text = article.text



**Cleaning Text**

In [3]:
text = re.sub(r"(\Wcrosstalk \d\d:\d\d:\d\d\W)","",text)
text = text.replace('Part 2','')
text = text.replace('\n\n\n\n','\n\n')
clean_text = text.split('\n\n')

**Identifying the pattern of the transcript, to distinguish speakers and creating a dataframe to hold speaker name, spoken sentences and the time of speach**

In [4]:
count = 0
for i in clean_text:
    print(f'{count}{i}')
    count += 1

0Chris Wallace: (01:20)
1Good evening from the Health Education Campus of Case Western Reserve University and the Cleveland Clinic. I’m Chris Wallace of Fox News and I welcome you to the first of the 2020 Presidential Debates between President Donald J. Trump and former Vice President Joe Biden. This debate is sponsored by the Commission on Presidential debates. The Commission has designed the format, six roughly 15 minute segments with two minute answers from each candidate to the first question, then open discussion for the rest of each segment. Both campaigns have agreed to these rules. For the record, I decided the topics and the questions in each topic. I can assure you none of the questions has been shared with the Commission or the two candidates.
2Chris Wallace: (02:10)
3This debate is being conducted under health and safety protocols designed by the Cleveland Clinic, which is serving as the Health Security advisor to the Commission for all four debates. As a precaution, both c

In [5]:
speaker = []
time = []
sentence = []
tmp = []
count = 0
for i in clean_text:
    if count%2 == 0 :
        tmp = i.split(': ')
        if (len(tmp) == 1):
            time.append('null')
            speaker.append(tmp[0])
        else:
            speaker.append(tmp[0])
            time.append(tmp[1])

    else:
        sentence.append(i)
    count += 1

print(len(speaker))
print(len(time))
print(len(sentence))

648
648
648


In [6]:
#consolidate dialogue for President Trump and vice President Biden
biden = ""
trump = ""
chris = ""
for i in range(0,len(speaker)):
    if (speaker[i] == 'Vice President Joe Biden'):
        biden += sentence[i]+ " "
    elif (speaker[i] == 'President Donald J. Trump'):
        trump += sentence[i]+ " "
    else:
        chris += sentence[i]+ " "
        
print(trump)

How are you doing? Thank you very much, Chris. I will tell you very simply. We won the election. Elections have consequences. We have the Senate, we have the White House, and we have a phenomenal nominee respected by all. Top, top academic, good in every way. Good in every way. In fact, some of her biggest endorsers are very liberal people from Notre Dame and other places. So I think she’s going to be fantastic. We have plenty of time. Even if we did it after the election itself. I have a lot of time after the election, as you know. So I think that she will be outstanding. She’s going to be as good as anybody that has served on that court. We really feel that. We have a professor at Notre Dame, highly respected by all, said she’s the single greatest student he’s ever had. He’s been a professor for a long time at a great school. And we won the election and therefore we have the right to choose her, and very few people knowingly would say otherwise. And by the way, the Democrats, they wo

In [8]:
df_test = pd.DataFrame()
df_test['speaker'] = speaker
df_test['time'] = time
df_test['sentence'] = sentence

df_test

,speaker,time,sentence
0,Chris Wallace,(01:20),Good evening from the Health Education Campus ...
1,Chris Wallace,(02:10),This debate is being conducted under health an...
2,Vice President Joe Biden,(02:49),"How you doing, man?"
3,President Donald J. Trump,(02:51),How are you doing?
4,Vice President Joe Biden,(02:51),I’m well.
...,...,...,...
643,President Donald J. Trump,(49:27),I want crystal clean water and air. I want bea...
644,Chris Wallace,(50:09),What do you believe about the science of clima...
645,President Donald J. Trump,(50:13),I believe that we have to do everything we can...
646,Chris Wallace,(50:25),"You believe that human pollution, gas, greenho..."


**Removing Modrators data from the dataframe**

In [49]:
df_candidate = df_candidate[df_candidate['speaker'] != 'Chris Wallace:']

In [9]:
df_candidate = df_test[df_test['speaker'] != 'Chris Wallace']


,speaker,time,sentence
2,Vice President Joe Biden,(02:49),"How you doing, man?"
3,President Donald J. Trump,(02:51),How are you doing?
4,Vice President Joe Biden,(02:51),I’m well.
6,President Donald J. Trump,(04:01),"Thank you very much, Chris. I will tell you ve..."
7,President Donald J. Trump,(04:53),And we won the election and therefore we have ...
...,...,...,...
639,Vice President Joe Biden,(48:45),Mitt Romney on that committee said it wasn’t w...
641,Vice President Joe Biden,(48:52),So would I.
643,President Donald J. Trump,(49:27),I want crystal clean water and air. I want bea...
645,President Donald J. Trump,(50:13),I believe that we have to do everything we can...


**Finding the polarity and subjectivity of the sentences used by both presidential Candidates**

In [33]:
import statistics as stats

In [43]:
pol_dict = {}
sub_dict ={}
for ind in df_candidate.index:
    temp_pol=[]
    temp_sub=[]
    blob = TextBlob(df_candidate["sentence"][ind])
    for sens in blob.sentences:
        temp_pol.append(sens.sentiment.polarity)
        temp_sub.append(sens.sentiment.subjectivity)
    pol_dict[ind] = round(stats.mean(temp_pol),2)
    sub_dict[ind] = round(stats.mean(temp_sub),2)
df_candidate["Polarity"] = pol_dict.values()
df_candidate["Subjectivity"] = sub_dict.values()

    
    

C:\Users\it09a\anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
C:\Users\it09a\anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [44]:
df_candidate

,speaker,time,sentence,Polarity,Subjectivity
2,Vice President Joe Biden,(02:49),"How you doing, man?",0.00,0.00
3,President Donald J. Trump,(02:51),How are you doing?,0.00,0.00
4,Vice President Joe Biden,(02:51),I’m well.,0.00,0.00
6,President Donald J. Trump,(04:01),"Thank you very much, Chris. I will tell you ve...",0.25,0.36
7,President Donald J. Trump,(04:53),And we won the election and therefore we have ...,0.04,0.30
...,...,...,...,...,...
639,Vice President Joe Biden,(48:45),Mitt Romney on that committee said it wasn’t w...,0.15,0.10
641,Vice President Joe Biden,(48:52),So would I.,0.00,0.00
643,President Donald J. Trump,(49:27),I want crystal clean water and air. I want bea...,0.22,0.43
645,President Donald J. Trump,(50:13),I believe that we have to do everything we can...,0.54,0.80


**Plotting the Polarity and subjectivity of the sentences spoken by both the presidential candidates**

In [50]:
fig = px.scatter(df_candidate,
                 x = 'Polarity',
                 y = 'Subjectivity',
                 color = 'speaker' ,
                 hover_data = ['time', 'sentence'],
                 title = 'Speach Analysis'
                )
fig.show()

In [53]:
fig = px.scatter(df_candidate,
                 x = 'Polarity',
                 y = 'Subjectivity',
                 color = 'speaker' ,
                 hover_data = ['time', 'sentence'],
                 animation_frame = 'time',
                 animation_group= 'speaker',
                 title = 'Speach Analysis through out debate'
                )
fig.show()